In [10]:
#Import Dependencies
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd 
from config import password


In [2]:
# Read in CSVs

filename = "PlayerBios.csv"
filepath = "../../DataBase/Resources/"
player_df = pd.read_csv(f"{filepath}{filename}")

      

In [3]:
#Check df columns
player_df.head()

,Player_ID,Name,rel_link,Position,height,weight (lbs),dob_age,experience,college,height.1,weight_lbs,dob,exp(y),draft_year,draft_round,draft_pick,draft_team
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0""",226,"June 30, 1997 (25)",4th Season,Mississippi,"6'0""",226,6/30/1997,4,2019,2.0,51.0,TEN
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4""",207,"July 31, 1988 (33)",12th Season,Georgia,"6'4""",207,7/31/1988,12,2011,1.0,4.0,CIN
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5""",245,"September 20, 1991 (30)",5th Season,Arkansas,"6'5""",245,9/20/1991,5,2015,6.0,202.0,NE
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0""",247,"May 2, 1998 (24)",3rd Season,Boston College,"6'0""",247,5/2/1998,3,2020,2.0,62.0,GB
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3""",215,"September 13, 1990 (31)",8th Season,Alabama,"6'3""",215,9/13/1990,8,2014,5.0,164.0,CIN


In [4]:
#drop duplicate columns
player_df.drop(columns="height.1", inplace = True)
player_df.drop(columns="weight_lbs", inplace = True)

In [5]:
#Create column for age 
player_df["age"] = player_df["dob_age"].str[-3:-1]
player_df.drop(columns="dob_age", inplace = True)
player_df.head()

,Player_ID,Name,rel_link,Position,height,weight (lbs),experience,college,dob,exp(y),draft_year,draft_round,draft_pick,draft_team,age
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0""",226,4th Season,Mississippi,6/30/1997,4,2019,2.0,51.0,TEN,25
1,12845,A.J. Green,/nfl/aj-green-fantasy/12845,WR,"6'4""",207,12th Season,Georgia,7/31/1988,12,2011,1.0,4.0,CIN,33
2,16962,AJ Derby,/nfl/aj-derby-fantasy/16962,TE,"6'5""",245,5th Season,Arkansas,9/20/1991,5,2015,6.0,202.0,NE,30
3,21802,AJ Dillon,/nfl/aj-dillon-fantasy/21802,RB,"6'0""",247,3rd Season,Boston College,5/2/1998,3,2020,2.0,62.0,GB,24
4,16116,AJ McCarron,/nfl/aj-mccarron-fantasy/16116,QB,"6'3""",215,8th Season,Alabama,9/13/1990,8,2014,5.0,164.0,CIN,31


In [6]:
#Function to apply to df in order to calculate height to float
def calc_ht(ht):
    height = ht.split("'")
    feet = float(height[0])
    inches = float(height[1].replace("\"",""))
    calc = feet + (inches / 12)
    return (calc)

In [7]:
#Apply function to height column
player_df["height (ft)"] = player_df['height'].apply(lambda x: calc_ht(x))
player_df.head(1)

,Player_ID,Name,rel_link,Position,height,weight (lbs),experience,college,dob,exp(y),draft_year,draft_round,draft_pick,draft_team,age,height (ft)
0,21042,A.J. Brown,/nfl/aj-brown-fantasy/21042,WR,"6'0""",226,4th Season,Mississippi,6/30/1997,4,2019,2.0,51.0,TEN,25,6.0


In [8]:
#format columns to lower case for db
player_df.columns = player_df.columns.str.lower()

Start SQL Imports

In [12]:
#Create connection to AWS database
ENDPOINT="fantasyfootball.ckswyhqaro8q.us-east-2.rds.amazonaws.com"
PORT="5432"
USR="DreamTeam"
PSSWD=password
DBNAME="postgres"

#conn_string = "host='localhost' dbname='my_database' user='postgres' password='secret'
con= create_engine("postgresql://{}:{}@{}:{}/{}".format(USR, PSSWD, ENDPOINT, PORT, DBNAME))


In [13]:
#Write table to AWS
player_df.to_sql("player_bios", con, if_exists="replace")